# Algoritmos para recorridos de grafos


La clase AdjacentVertex representa una tupla donde el primer elemento es un vértice y el segundo el peso asociado. 



In [1]:
class AdjacentVertex:
    """ This class allows us to represent a tuple
    with an adjacent vertex
    and the weight associated (by default None, for non-unweighted graphs)"""
    def __init__(self, vertex: object, weight: int = 1) -> None:
        self.vertex = vertex
        self.weight = weight

    def __eq_(self, other: 'AdjacentVertex') -> bool:
        if other is None: 
            return False
        return self.vertex == other.vertex and self.weight == other.weight 
        
    def __str__(self) -> str:
        """ returns the tuple (vertex, weight)"""
        if self.weight is not None:
            return '(' + str(self.vertex) + ',' + str(self.weight) + ')'
        else:
            return str(self.vertex)

In [2]:

class Graph:
    def __init__(self, vertices: list, directed: bool = True) -> None:
        """ We use a dictionary to represent the graph
        the dictionary's keys are the vertices
        The value associated for a given key will be the list of their neighbours.
        Initially, the list of neighbours is empty"""
        self._vertices = {}
        for v in vertices:
            self._vertices[v] = []
        self._directed = directed

    def add_vertex(self, vertex: str) -> None:
        if vertex in self._vertices.keys():
            print(vertex, ' already exists!')
            return
        self._vertices[vertex] = []

    def add_edge(self, start: object, end: object, weight: int = 1) -> None:
        if start not in self._vertices.keys():
            print(start, ' does not exist!')
            return
        if end not in self._vertices.keys():
            print(end, ' does not exist!')
            return

        # adds to the end of the list of neighbours for start
        self._vertices[start].append(AdjacentVertex(end, weight))

        if not self._directed:
            # adds to the end of the list of neighbors for end
            self._vertices[end].append(AdjacentVertex(start, weight))

    def contain_edge(self, start: object, end: object) -> int:
        """ checks if the edge (start, end) exits. It does
        not exist return 0, eoc returns its weight or 1 (for unweighted graphs)"""
        if start not in self._vertices.keys():
            print(start, ' does not exist!')
            return 0
        if end not in self._vertices.keys():
            print(end, ' does not exist!')
            return 0

        # we search the AdjacentVertex whose v is equal to end

        for adj in self._vertices[start]:
            if adj.vertex == end:
                return adj.weight

        return 0  # does not exist

    def remove_edge(self, start: object, end: object):
        """ removes the edge (start, end)"""
        if start not in self._vertices.keys():
            print(start, ' does not exist!')
            return
        if end not in self._vertices.keys():
            print(end, ' does not exist!')
            return

        # we must look for the adjacent AdjacentVertex (neighbour)  whose vertex is end, and then remove it
        exist = False
        for adj in self._vertices[start]:
            if adj.vertex == end:
                exist = True
                self._vertices[start].remove(adj)
        if not self._directed:
            # we must also look for the AdjacentVertex (neighbour)  whose vertex is end, and then remove it
            for adj in self._vertices[end]:
                if adj.vertex == start:
                    self._vertices[end].remove(adj)

        if not exist: 
            print("({},{}) does not exist!!!!".format(start, end))


    def __str__(self) -> str:
        """ returns a string containing the graph"""
        result = ''
        for v in self._vertices:
            result += '\n'+str(v)+':'
            for adj in self._vertices[v]:
                result += str(adj)+"  "
        result += '\n'
        return result

    def get_adjacent_vertices(self, start: object) -> list:
        """ returns a Python list containing the adjacent
        vertices of vertex. The list only contains the vertices"""
        if start not in self._vertices.keys():
            print(start, ' does not exist!')
            return None
        
        result = []
        for adj in self._vertices[start]:
            result.append(adj.vertex)
        return result


    def get_origins(self, end: object) -> list:
        """ returns a Python list containing those vertices that have
        an edge to vertex. The list is formed with objects of AdjacentVertex"""
        if end not in self._vertices.keys():
            print(end, ' does not exist!')
            return None
        
        result = []
        for v in self._vertices.keys():
            for adj in self._vertices[v]:
                if adj.vertex == end:
                    result.append(v)

        return result
  


Vamos a extender Graph para que contenga los recorridos dfs y bfs:

In [3]:
class Graph2(Graph):

    def bfs(self, start: object) -> list:
        """returns a list with the DFS traversal from start"""
        # first, we check that start is a vertex in the graph
        if start not in self._vertices.keys():
            print(start, ' does not exist!!!')
            return None
        # list to save the BFS traversal from start
        result = []
        # dictionary to mark those vertices that we have already added to the queue 
        visited = dict.fromkeys(self._vertices.keys(), False)

        q = []  # we use a list as a queue (we add to the end, and get from the beginning)
        # add to the queue
        q.append(start)
        # we visite the first element
        visited[start] = True

        while len(q) > 0 :  # while the queue is not empty
            v = q.pop(0)    # we get the first element
            result.append(v)    # we add it into result with the BFS traversal

            # we have to get its adjacent vertices
            lst_adj = self.get_adjacent_vertices(v)
            # we visite them
            for v in lst_adj:
                # if an adjacent vertex has not been visited yet, we must add it to the queue and mark it as visited
                if not visited[v]:
                    visited[v] = True   # mark it as visited
                    q.append(v)         # enqueue

        # finally, we return the BFS traversal
        return result

    
    def dfs(self, start: object) -> list:
        """returns a list with the DFS traversal from start"""
        # first, we check that start is a vertex in the graph
        if start not in self._vertices.keys():
            print(start, ' does not exist!!!')
            return None
        # list to save the DFS traversal from start
        result = []
        # dictionary to mark those vertices that have been already visited
        visited = dict.fromkeys(self._vertices.keys(), False)
        # we call to the recursive method _dfs
        self._dfs(start, visited, result)
        # return the DFS traversal
        return result

    
    def _dfs(self, start: object, visited: dict, result: list) -> None:
        """recursive method to obtain the DFS traversal from start"""
        # firstly, we mark start as visited and add it to result (containing the DFS traversal)
        visited[start] = True
        result.append(start)
        # now, we gets the adjacent vertices of start
        for v in self.get_adjacent_vertices(start):
            # we only call the recursive function for those vertices that have not been visited yet
            if not visited[v]:
                self._dfs(v, visited, result)
            

    def dfs_ite(self, start: object) -> list:
        """iterative version of the DFS traversal. This use a stack to store the previous vertices 
        """
        if start not in self._vertices.keys():
            print(start, ' does not exist!!!')
            return None
        # create the list that will save the DFS traversal from start
        result = []
        # we create a dictionary to know what vertices have been already visited
        # At the beginning, all the vertices must have value False (not visited)
        # When the vertex is added to result, then this must be marked as visited
        visited = dict.fromkeys(self._vertices.keys(), False)
        
        # The stack helps us to save the adjacent vertices that have not been visited yet.
        # start will be the first element in the stack to begin with start
        stack = []      # it is a list that is used as a stack (we add to the end and always get its last element)
        stack.append(start)

        while len(stack)>0:
            last = stack.pop() # gets and removes the last element that was added to the stack

            if not visited[last]:
                # if the vertex last has not been visited yet,  
                result.append(last) # we must add it to the DFS traversal
                visited[last] = True    # also, we mark it as visited
            
            # Regardless of whether the vertex last had already been visited or not, 
            # we must traverse all of its adjacent vertices
            for v in reversed(self.get_adjacent_vertices(last)):    # we need to use reversed to obtain the same DFS traversal than using the recurvise version
                # if the adjacent vertex has not been visited yet, we add it to the stack
                if not visited[v]:
                    stack.append(v)

        return result


<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/f/fe/Tred-G.svg/440px-Tred-G.svg.png' width='300'>


In [4]:
labels = ['a', 'b', 'c', 'd', 'e']
g = Graph2(labels)

g.add_edge('a','b')
g.add_edge('a','c')
g.add_edge('a','d')
g.add_edge('a','e')

g.add_edge('b','d')
g.add_edge('c','d')
g.add_edge('c','e')
g.add_edge('d','e')
# print(g)

# g.add_vertex('f')
# g.add_vertex('g')
# g.add_edge('f','g')
# print(g)



for v in g._vertices.keys():
    print("Bfs traversal for {}: {}".format(v, g.bfs(v)))
    print("dfs traversal for {}: {}".format(v, g.dfs(v)))
    print("dfs based on stack traversal for {}: {}".format(v, g.dfs_ite(v)))
    print()

Bfs traversal for a: ['a', 'b', 'c', 'd', 'e']
dfs traversal for a: ['a', 'b', 'd', 'e', 'c']
dfs based on stack traversal for a: ['a', 'b', 'd', 'e', 'c']

Bfs traversal for b: ['b', 'd', 'e']
dfs traversal for b: ['b', 'd', 'e']
dfs based on stack traversal for b: ['b', 'd', 'e']

Bfs traversal for c: ['c', 'd', 'e']
dfs traversal for c: ['c', 'd', 'e']
dfs based on stack traversal for c: ['c', 'd', 'e']

Bfs traversal for d: ['d', 'e']
dfs traversal for d: ['d', 'e']
dfs based on stack traversal for d: ['d', 'e']

Bfs traversal for e: ['e']
dfs traversal for e: ['e']
dfs based on stack traversal for e: ['e']



<img src='https://static.javatpoint.com/ds/images/breadth-first-search-algorithm-example.png' widht='300'>

In [5]:
labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
g = Graph2(labels)

g.add_edge('A','B')
g.add_edge('A','D')

g.add_edge('B','C')
g.add_edge('B','F')

g.add_edge('C','E')
g.add_edge('C','G')

g.add_edge('D','F')

g.add_edge('E','F')
g.add_edge('E','B')

g.add_edge('F','A')

g.add_edge('G','E')

g.add_vertex('H')
g.add_vertex('I')
g.add_vertex('J')

g.add_edge('H', 'I')
g.add_edge('H', 'J')


print(g)

for v in g._vertices.keys():
    print("Bfs traversal for {}: {}".format(v, g.bfs(v)))
    print("dfs traversal for {}: {}".format(v, g.dfs(v)))
    print("dfs traversal iterative for {}: {}".format(v, g.dfs_ite(v)))

    print()



A:(B,1)  (D,1)  
B:(C,1)  (F,1)  
C:(E,1)  (G,1)  
D:(F,1)  
E:(F,1)  (B,1)  
F:(A,1)  
G:(E,1)  
H:(I,1)  (J,1)  
I:
J:

Bfs traversal for A: ['A', 'B', 'D', 'C', 'F', 'E', 'G']
dfs traversal for A: ['A', 'B', 'C', 'E', 'F', 'G', 'D']
dfs traversal iterative for A: ['A', 'B', 'C', 'E', 'F', 'G', 'D']

Bfs traversal for B: ['B', 'C', 'F', 'E', 'G', 'A', 'D']
dfs traversal for B: ['B', 'C', 'E', 'F', 'A', 'D', 'G']
dfs traversal iterative for B: ['B', 'C', 'E', 'F', 'A', 'D', 'G']

Bfs traversal for C: ['C', 'E', 'G', 'F', 'B', 'A', 'D']
dfs traversal for C: ['C', 'E', 'F', 'A', 'B', 'D', 'G']
dfs traversal iterative for C: ['C', 'E', 'F', 'A', 'B', 'D', 'G']

Bfs traversal for D: ['D', 'F', 'A', 'B', 'C', 'E', 'G']
dfs traversal for D: ['D', 'F', 'A', 'B', 'C', 'E', 'G']
dfs traversal iterative for D: ['D', 'F', 'A', 'B', 'C', 'E', 'G']

Bfs traversal for E: ['E', 'F', 'B', 'A', 'C', 'D', 'G']
dfs traversal for E: ['E', 'F', 'A', 'B', 'C', 'G', 'D']
dfs traversal iterative for E: ['E'

# Ejercicio: 
1) Crea el siguiente grafo no dirigido y obtén sus recorridos a partir de cada uno de sus vértices: 
<img src='https://graphonline.ru/tmp/saved/XR/XRmBpPjiLYfUvjSa.png'>
2) Modifica el grafo anterior para que quede como el siguiente y obtén sus recorridos a partir de cada uno de sus vértices: 
<img src='https://graphonline.ru/tmp/saved/ep/eprzBsvpSiOpDvjj.png'>
3) Crea el siguiente grafo ponderado y y obtén sus recorridos a partir de cada uno de sus vértices:
<img src='https://infinitegraph.com/wp-content/uploads/2021/04/WeightedGraph01.png'>

4) Práctica con otros grafos. 